In [1]:
import requests
from datetime import datetime, timedelta
from pprint import pprint
import pandas as pd
import matplotlib.pyplot as plt
from draw_map import  BORDER_LINES, drawKorea, drawKoreaMinus

In [2]:
# 측정소별 실시간 측정정보 조회 (매시 15분 내외 업데이트)
base_url = "http://apis.data.go.kr/B552584/ArpltnInforInqireSvc/getMsrstnAcctoRltmMesureDnsty"
with open('keys/에어코리아api.txt') as file:
    service_key = file.read()       # 발급받은 에어코리아 API 키 입력

In [3]:
import requests
import pandas as pd

# 예제 데이터프레임 생성
station_Name = pd.read_csv('data/전국 대기측정소_완료.csv', encoding='utf-8')

# 완성코드
def get_air_quality():
    # 결과를 저장할 리스트
    results_list = []
    
    # 각 측정소에 대한 데이터 수집
    for index, row in station_Name.iterrows():
        # 웹 요청시 같이 전달될 데이터 = 요청 메시지
        params = {
            'serviceKey' : service_key,
            'returnType' : 'JSON',
            'numOfRows' : 30,
            'pageNo' : 1,
            'stationName' : row['측정소명'],  # 각 행의 측정소명을 선택
            'dataTerm' : 'DAILY',
            'ver' : "1.4"  # 오퍼레이션 버전
        }

        res = requests.get(url=base_url , params=params)

        # 응답 데이터 정리
        data = res.json()['response']['body']['items']
        
        # 데이터가 존재하면 처리
        
            # 데이터를 날짜와 시간에 대한 기준으로 정렬
        sorted_data = sorted(data, key=lambda x: x['dataTime'], reverse=True)

        # 최신 데이터 선택
        latest_data = sorted_data[0]

        # 최종 데이터 생성
        result = {
            '날짜': latest_data['dataTime'],
            '이름': latest_data['stationName'],
            '측정망 정보': latest_data['mangName'],
            '아황산가스 농도': latest_data['so2Value'] + "ppm",
            '일산화탄소 농도': latest_data['coValue'] + "ppm",
            '오존 농도': latest_data['o3Value'] + "ppm",
            '이산화질소 농도': latest_data['no2Value'] + "ppm",
            '미세먼지(PM10) 농도': latest_data['pm10Value'] + "ug/m³",
            '초미세먼지(PM2.5) 농도': latest_data['pm25Value'] + "ug/m³"
        }

        # 결과 리스트에 추가
        results_list.append(result)
    
    return results_list


pop = pd.read_csv('data/카토그램.csv', encoding='utf-8')

pop['미세먼지(PM10) 농도'] = get_air_quality()[0]['미세먼지(PM10) 농도']
pop['미세먼지(PM10) 농도'] = pop['미세먼지(PM10) 농도'].str.extract('(\d+)').astype(float)

drawKorea('미세먼지(PM10) 농도', pop, 'Blues')

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
pop.tail(100)

,Unnamed: 0,ID,광역시도,시군구,격자 X,격자 Y,y,x,미세먼지(PM10) 농도
150,150,익산,전라북도,익산시,60,91,16,3,20.0
151,151,정읍,전라북도,정읍시,58,83,18,2,20.0
152,152,남원,전라북도,남원시,68,80,18,5,20.0
153,153,김제,전라북도,김제시,59,88,17,3,20.0
154,154,완주,전라북도,완주군,63,89,17,4,20.0
...,...,...,...,...,...,...,...,...,...
245,245,철원,강원특별자치도,철원군,65,139,0,7,20.0
246,246,화천,강원특별자치도,화천군,72,139,0,8,20.0
247,247,양구,강원특별자치도,양구군,77,139,0,9,20.0
248,248,인제,강원특별자치도,인제군,80,138,1,9,20.0
